In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import spherical_jn, lpmv


# -------------------------------------------------------
# Paramètres physiques
# -------------------------------------------------------
f = 40000          # fréquence (Hz)
c0 = 343           # vitesse du son dans l'air (m/s)
ω = 2 * np.pi * f
k = ω / c0         # nombre d'onde

A = 1.0            # amplitude du vortex
l = 6       # ordre du vortex (radial)
m = 1              # moment angulaire (nombre topologique)
t = 0              # temps instantané pour la visualisation


# -------------------------------------------------------
# Espace de simulation
# -------------------------------------------------------
N =8 # nombre de transducteur par axe (8x8)
a=0.01

# bornes asymétriques : +/-5 cm latéralement, 10 cm en hauteur
Lx = 0.05
Ly = 0.05

x = np.linspace(-Lx, Lx, N)
y = np.linspace(-Ly, Ly, N)
z = np.array([0]*(N**2))

X_T, Y_T = np.meshgrid(x, y)
Z_T=np.array([[0]*N]*N)

##on passe tout à une dimension, dans le ref de la plaque :
X_T=X_T.flatten()
Y_T=Y_T.flatten()
Z_T=Z_T.flatten()

# ---- Constantes physiques ----
k = 2 * np.pi * 40000 / 343
R_p = 0.0005
c_p = 2400
rho_p = 28.59
c_0 = 343
rho_0 = 1.204
R_t = 0.005  # rayon transducteur

eps = 1e-4  # pas de dérivation

In [2]:
def compute_vortex_phases(xf1, yf1, zf1, xf2, yf2, zf2):

    ## dans le ref du vortex 1:

    Z_T_1=Z_T-zf1
    X_T_1=X_T-xf1
    Y_T_1=Y_T-yf1

    ## dans le ref du vortex 2:

    Z_T_2=Z_T-zf2
    X_T_2=X_T-xf2
    Y_T_2=Y_T-yf2

    # -------------------------------------------------------
    # Passage en coordonnées sphériques
    # -------------------------------------------------------
    r_1 = np.sqrt(X_T_1**2 + Y_T_1**2 + Z_T_1**2)
    θ_1 = np.arccos(Z_T_1/ (r_1 + 1e-12))  # angle polaire
    φ_1 = np.arctan2(Y_T_1, X_T_1)            # angle azimutal

    r_2 = np.sqrt(X_T_2**2 + Y_T_2**2 + Z_T_2**2)
    θ_2 = np.arccos(Z_T_2/ (r_2 + 1e-12))  # angle polaire
    φ_2 = np.arctan2(Y_T_2, X_T_2)            # angle azimutal

    # -------------------------------------------------------
    # Calcul du champ acoustique vortex(r,θ,φ) 1 et 2
    # -------------------------------------------------------
    jl_1 = spherical_jn(l, k * r_1)
    Plm_1 = lpmv(m, l, np.cos(θ_1))

    jl_2 = spherical_jn(l, k * r_2)

    Plm_2 = lpmv(m, l, np.cos(θ_2))

    vortex_1 = A * jl_1 * Plm_1 * np.exp(1j * (m * φ_1 - ω * t))
    vortex_2 = A * jl_2 * Plm_2 * np.exp(1j * (m * φ_2 - ω * t))

    vortex_tot=vortex_1+vortex_2

    # -------------------------------------------------------
    # Amplitude et phase
    # -------------------------------------------------------
    ampl = np.abs(vortex_tot)
    phase = np.angle(vortex_tot)
    phase=np.mod(phase,2*np.pi)

    # normalisation amplitude (comme tu fais déjà)
    ampl = ampl / np.max(ampl)

    return ampl, phase


In [3]:
# === Point de focalisation ===
X_foc1, Y_foc1, Z_foc1 = 0.01, 0, 0.03
X_foc2, Y_foc2, Z_foc2 = -0.01, 0, 0.03

# === Lancement ===
compute_vortex_phases(X_foc1, Y_foc1, Z_foc1, X_foc2, Y_foc2, Z_foc2)

(array([0.06419873, 0.09820753, 0.03223305, 0.02920379, 0.02920379,
        0.03223305, 0.09820753, 0.06419873, 0.09361655, 0.22970205,
        0.45049347, 0.31473269, 0.31473269, 0.45049347, 0.22970205,
        0.09361655, 0.01606383, 0.31750056, 0.53985366, 0.2921005 ,
        0.2921005 , 0.53985366, 0.31750056, 0.01606383, 0.01842117,
        0.38646633, 0.54557216, 1.        , 1.        , 0.54557216,
        0.38646633, 0.01842117, 0.01842117, 0.38646633, 0.54557216,
        1.        , 1.        , 0.54557216, 0.38646633, 0.01842117,
        0.01606383, 0.31750056, 0.53985366, 0.2921005 , 0.2921005 ,
        0.53985366, 0.31750056, 0.01606383, 0.09361655, 0.22970205,
        0.45049347, 0.31473269, 0.31473269, 0.45049347, 0.22970205,
        0.09361655, 0.06419873, 0.09820753, 0.03223305, 0.02920379,
        0.02920379, 0.03223305, 0.09820753, 0.06419873]),
 array([3.61093184, 3.97151321, 4.18263838, 2.24921549, 0.89237716,
        5.24213958, 5.45326475, 5.81384612, 3.76151793, 4.